In [ ]:
from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/MyDrive/Colab Notebooks

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#依照所需修改下面參數:
# 1.資料路徑
DATASET_PATH  = 'Two-Class-ori-1/Image_result'
# 2.影像大小
IMAGE_SIZE = (227,227) #(224,224) (227,227) (299,299)

# 3.影像類別數
NUM_CLASSES = 2 # 5

# 4.若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 16

# 5.Epoch數
NUM_EPOCHS = 84

# 6.模型輸出儲存的檔案
WEIGHTS_FINAL = 'Two-Class-ori-1/Image_result/test5.h5'

#讀取路徑下圖片
train_datagen = ImageDataGenerator(rescale=1./255)
train_batches = train_datagen.flow_from_directory('Two-Class-ori-1/Image_result/',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE) #color_mode='grayscale',

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_batches = valid_datagen.flow_from_directory('Two-Class-ori-1/valid/',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)
#print(DATASET_PATH + 'train\\')
print(train_batches.image_shape)
print(valid_batches.image_shape)

In [ ]:
#使用Callbacks，使訓練模型Early Stop
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

#為了防止overfitting，在10個epoch內 val_loss數值都沒有下降時會停止模型的訓練
earlystop = EarlyStopping(patience=10) #callback要設定

#Learning Rate Reduction 當訓練已無改善時，降低學習率

#googlenet模型: monitor =val_auxilliary_output_2_accuracy
#其他分類模型 : monitor =val_accuracy
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_auxilliary_output_2_accuracy',   
                                            patience = 2,#代表模型連續訓練2個epoch後，監控的數值表現沒有更好，就會停止訓練。
                                            verbose = 1, #用於選擇模型訓練資訊的顯示方式。
                                            factor = 0.5,#缩放學習率的常數。訓練模型中，若符合patience條件
                                            #(如：連續訓練5個epoch後，val_loss未降低)，則將原本的學習率值乘以factor，做為新的學習率。
                                            min_lr = 0.00001#每當符合patience條件，學習率會變動(縮小)，min_lr代表學習率的下限。
                                            )

#模型效果變好就保存，最後儲存最好的模型
filepath="'Two-Class-ori/models/best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_auxilliary_output_2_accuracy', verbose=1, save_best_only=True,mode='max')

callbacks = [learning_rate_reduction,checkpoint]

In [ ]:
#建立模型
#Alexnet (image size 227x227)

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout,Conv2D, MaxPooling2D

# 建立模型序列
model = Sequential()
#第一層卷積網路，使用96個卷積核，大小為11x11步長為4， 要求輸入的圖片為227x227， 3個通道，不加邊，啟用函式使用relu
model.add(Conv2D(96, (11, 11), strides=(4, 4), input_shape=(227, 227, 3), padding='valid', activation='relu',kernel_initializer='uniform'))
# 池化層
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
# 第二層加邊使用256個5x5的卷積核，加邊，啟用函式為relu
model.add(Conv2D(256, (5, 5), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
#使用池化層，步長為2
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
# 第三層卷積，大小為3x3的卷積核使用384個
model.add(Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
# 第四層卷積,同第三層
model.add(Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
# 第五層卷積使用的卷積核為256個，其他同上
model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy']) #binary_crossentropy

model.summary()



In [ ]:
# 訓練模型
History=model.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                        class_weight={0:0.850174216,1:1.213930348},
                        callbacks = callbacks) 

#2 class: class_weight={0:0.850174216,1:1.213930348}
#5 class: class_weight={0:0.346844,1:1.008696,2:2.109091,3:3.538983,4:2.711688}

# 儲存訓練好的模型
model.save(WEIGHTS_FINAL)